In [1]:
## ca default dispute
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import gc
import re
import time
import datetime
import xml.etree.ElementTree as ET

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

import xgboost as xgb
import lightgbm as lgbm

pd.set_option("display.max_columns",None)

os.chdir("D:/Datasets/Brainwaves/ML/")
train_labels=pd.read_csv("train_labels.csv")
print(train_labels.shape)
train_labels

plt.xticks(rotation='90')
column_name='CA_fallback_default_dispute'
train_labels.loc[train_labels[column_name]=='third party in case of default ',column_name]='third party in case of default'
train_labels.loc[train_labels[column_name]==' third party in case of default',column_name]='third party in case of default'
train_labels[column_name].value_counts()
print(train_labels[column_name].value_counts())
# sns.countplot(train_labels[column_name])
# plt.show()

correction={"NotFound":0,
 "third party in case of default":1,
 "non defaulting party in case of default":2,
 "substitution calculation agent in case of dispute":3,
 "substitution calculation agent in case of default":4,
 "other in case of dispute":5,
 "substitution calculation agent in case of dispute;other in case of dispute":3,
 "non defaulting party in case of default;third party in case of default":2,
 "Substitution calculation agent in case of dispute":3,
 "substitution calculation agent in case of default;substitution calculation agent in case of dispute":4,
 'third party in case of default;substitution calculation agent in case of dispute ':1,
 ' non defaulting party in case of default ':2,
' substitution calculation agent in case of dispute':3,
 ' non defaulting party in case of dispute;substitution calculation agent in case of dispute ':2,
 'non defaulting party in case of default;non defaulting party in case of dispute ':2,
 'substitution calculation agent in case of default;third party in case of default':3
}
train_labels[column_name]=train_labels[column_name].map(correction).fillna(0)
train_labels[column_name].value_counts()

filenames=[name for name in os.listdir("data/") if ".xml" in name]
features=pd.DataFrame()
c=0
features['File_Name']=["NotFound"]*len(filenames)
features['Text']=0
for i,filename in enumerate(filenames):
#     print(filename,":")RD_party_A_SnP_short_term_trigger_method
    tree=ET.parse("data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        print(filename)
        features.loc[i,'File_Name']="19_bk"
    else:
        features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"par"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if "calculation agent" in elem.text.lower():
#                 print(elem.text)
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
    features.loc[i,'Text']=data

features=features.merge(right=train_labels[['File_Name',column_name]],how='left',on=['File_Name'])
print(features.shape)
features.head()



encoder={"NotFound":0,
 "third party in case of default":1,
 "non defaulting party in case of default":2,
 "substitution calculation agent in case of dispute":3,
 "substitution calculation agent in case of default":4,
 "other in case of dispute":5}
decoder=dict(zip(encoder.values(),encoder.keys()))

tfidf=TfidfVectorizer(lowercase=True,stop_words='english',max_features=1000)
tfidf.fit(features['Text'])

X=tfidf.transform(features['Text']).toarray()
y=features[column_name].values
print(X.shape,y.shape)


y[y==0].shape[0]/y.shape[0]

X_train,X_val,y_train,y_val=train_test_split(X,y,stratify=y,test_size=0.25,random_state=42)

print("Training Shape",X_train.shape,y_train.shape)
print("Validation Shape",X_val.shape,y_val.shape)

lr=LogisticRegression(solver="liblinear")
lr.fit(X_train,y_train)

print("Training Score",lr.score(X_train,y_train))
print("Validation Score",lr.score(X_val,y_val))

xgbclf=xgb.XGBClassifier()

xgbclf.fit(X_train,y_train)

print("Training Score",xgbclf.score(X_train,y_train))
print("Validation Score",xgbclf.score(X_val,y_val))

test_filenames=[name for name in os.listdir("public_test_data/") if ".xml" in name]
test_features=pd.DataFrame()
c=0
test_features['File_Name']=["NotFound"]*len(test_filenames)
test_features['Text']=0
for i,filename in enumerate(test_filenames):
#     print(filename,":")
    tree=ET.parse("public_test_data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        print(filename)
        test_features.loc[i,'File_Name']="19_bk"
    else:
        test_features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"par"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if "calculation agent" in elem.text.lower():
#                 print(elem.text)
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
    test_features.loc[i,'Text']=data

print(test_features.shape)
test_features.head()

X_test=tfidf.transform(test_features['Text'].values)
print("Testing Shape",X_test.shape)

sub=pd.DataFrame()
sub['File_Name']=test_features['File_Name']
sub[column_name]=pd.Series(xgbclf.predict(X_test)).map(decoder)
sub.head()

sub.to_csv("public test predictions/"+column_name+".csv",index=False)
print("Done")

## ca dispute resolution
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import gc
import re
import time
import datetime
import xml.etree.ElementTree as ET

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

import xgboost as xgb
import lightgbm as lgbm

pd.set_option("display.max_columns",None)

train_labels=pd.read_csv("train_labels.csv")
print(train_labels.shape)
train_labels

plt.xticks(rotation='90')
column_name='CA_dispute_resolution '
print(train_labels[column_name].value_counts())
# sns.countplot(train_labels[column_name])
# plt.show()

correction={"Yes":"yes","YES":"yes","yes":"yes","No":"no","NO":"no","no":"no","NotFound":"no"}
train_labels[column_name]=train_labels[column_name].map(correction)

filenames=[name for name in os.listdir("data/") if ".xml" in name]
features=pd.DataFrame()
c=0
features['File_Name']=["NotFound"]*len(filenames)
features['Text']=0
for i,filename in enumerate(filenames):
#     print(filename,":")RD_party_A_SnP_short_term_trigger_method
    tree=ET.parse("data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        print(filename)
        features.loc[i,'File_Name']="19_bk"
    else:
        features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"par"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if "calculation agent" in elem.text.lower():
#                 print(elem.text)
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
    features.loc[i,'Text']=data

features=features.merge(right=train_labels[['File_Name',column_name]],how='left',on=['File_Name'])
print(features.shape)
features.head()

encoder={"yes":1,"no":0}
decoder={1:"yes",0:"no"}

tfidf=TfidfVectorizer(lowercase=True,stop_words='english',max_features=1000)
tfidf.fit(features['Text'])

X=tfidf.transform(features['Text']).toarray()
y=features[column_name].map(encoder).values
print(X.shape,y.shape)

y[y==0].shape[0]/y.shape[0]

X_train,X_val,y_train,y_val=train_test_split(X,y,stratify=y,test_size=0.25,random_state=42)

print("Training Shape",X_train.shape,y_train.shape)
print("Validation Shape",X_val.shape,y_val.shape)

lr=LogisticRegression(solver="liblinear")
lr.fit(X_train,y_train)

print("Training Score",lr.score(X_train,y_train))
print("Validation Score",lr.score(X_val,y_val))

xgbclf=xgb.XGBClassifier()

xgbclf.fit(X_train,y_train)

print("Training Score",xgbclf.score(X_train,y_train))
print("Validation Score",xgbclf.score(X_val,y_val))

test_filenames=[name for name in os.listdir("public_test_data/") if ".xml" in name]
test_features=pd.DataFrame()
c=0
test_features['File_Name']=["NotFound"]*len(test_filenames)
test_features['Text']=0
for i,filename in enumerate(test_filenames):
#     print(filename,":")
    tree=ET.parse("public_test_data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        print(filename)
        test_features.loc[i,'File_Name']="19_bk"
    else:
        test_features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"par"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if "calculation agent" in elem.text.lower():
#                 print(elem.text)
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
    test_features.loc[i,'Text']=data

print(test_features.shape)
test_features.head()

X_test=tfidf.transform(test_features['Text'].values)
print("Testing Shape",X_test.shape)

sub=pd.DataFrame()
sub['File_Name']=test_features['File_Name']
sub[column_name]=pd.Series(xgbclf.predict(X_test)).map(decoder)
sub.head()

## governing law
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import gc
import re
import time
import datetime
import xml.etree.ElementTree as ET

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.svm import LinearSVC

import xgboost as xgb
import lightgbm as lgbm

# from imblearn.over_sampling import RandomOverSampler


pd.set_option("display.max_columns",None)


# In[ ]:


train_labels=pd.read_csv("train_labels.csv")
print(train_labels.shape)
train_labels


# In[ ]:


train_labels['governing law'].value_counts()


# In[ ]:


correction={"English Law":"English Law",
            
            "English law":"English Law",
           
            "Laws of the State of New York":"Laws of the State of New York",
           
            "Laws of England":"Laws of England",
           
            "The law of state of New york":"Laws of the State of New York",
           
            "Laws of the state of New York":"Laws of the State of New York",
           
            "Laws of the state of New york":"Laws of the State of New York",
           
            "Laws of the state of new york":"Laws of the State of New York",
           
            "Laws of New South Wales":"Laws of New South Wales",
           
            "NotFound":"NotFound",
           
            "Laws of the Hong Kong Special Administrative region of the People's Republic of China ":"NotFound",
           
            "Laws of England and Wales":"NotFound",
           
            "State of New South Wales":"Laws of New South Wales",
           
            "The Laws in New south wales":"Laws of New South Wales"}


column_name="governing law"

train_labels[column_name]=train_labels[column_name].map(correction)


# In[ ]:


filenames=[name for name in os.listdir("data/") if ".xml" in name]
features=pd.DataFrame()
c=0
features['File_Name']=["NotFound"]*len(filenames)
features['Text']=0
for i,filename in enumerate(filenames):
#     print(filename,":")RD_party_A_SnP_short_term_trigger_method
    tree=ET.parse("data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        print(filename)
        features.loc[i,'File_Name']="19_bk"
    else:
        features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"text"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if "governing law" in elem.text.lower():
#                 print(elem.text)
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
    features.loc[i,'Text']=data

features=features.merge(right=train_labels[['File_Name',column_name]],how='left',on=['File_Name'])
print(features.shape)
features.head()


# In[ ]:


features['governing law'].value_counts()


# In[ ]:


plt.xticks(rotation='90')
sns.countplot(features['governing law'])


# In[ ]:


encoder={"English Law":0,
 "Laws of the State of New York":1,
 "Laws of England":2,
 "NotFound":3,
 "Laws of New South Wales":4}
decoder=dict(zip(encoder.values(),encoder.keys()))

tfidf=TfidfVectorizer(lowercase=True,stop_words='english',max_features=1000)
tfidf.fit(features['Text'])

X=tfidf.transform(features['Text']).toarray()
y=features[column_name].map(encoder).values
print(X.shape,y.shape)


# In[ ]:


y[y==0].shape[0]/y.shape[0]


# In[ ]:





# In[ ]:


X_train,X_val,y_train,y_val=train_test_split(X,y,stratify=y,test_size=0.25,random_state=42)

print("Training Shape",X_train.shape,y_train.shape)
print("Validation Shape",X_val.shape,y_val.shape)


# In[ ]:


lr=LogisticRegression(solver="liblinear",multi_class='ovr')
lr.fit(X_train,y_train)

print("Training Score",lr.score(X_train,y_train))
print("Validation Score",lr.score(X_val,y_val))


# In[ ]:


xgbclf=xgb.XGBClassifier()

xgbclf.fit(X_train,y_train)

print("Training Score",xgbclf.score(X_train,y_train))
print("Validation Score",xgbclf.score(X_val,y_val))


# In[ ]:


test_filenames=[name for name in os.listdir("public_test_data/") if ".xml" in name]
test_features=pd.DataFrame()
c=0
test_features['File_Name']=["NotFound"]*len(test_filenames)
test_features['Text']=0
for i,filename in enumerate(test_filenames):
#     print(filename,":")
    tree=ET.parse("public_test_data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        print(filename)
        test_features.loc[i,'File_Name']="19_bk"
    else:
        test_features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"text"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if "governing law" in elem.text.lower():
#                 print(elem.text)
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
    test_features.loc[i,'Text']=data

print(test_features.shape)
test_features.head()


# In[ ]:


X_test=tfidf.transform(test_features['Text'].values)
print("Testing Shape",X_test.shape)


# In[ ]:


sub=pd.DataFrame()
sub['File_Name']=test_features['File_Name']
sub[column_name]=pd.Series(lr.predict(X_test)).map(decoder)
sub.head()


# In[ ]:


sns.countplot(sub[column_name])


# In[ ]:


sub.to_csv("public test predictions/"+column_name+".csv",index=False)
print("Done")

# rd_party_A_Fitch_long_term_trigger_method
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import gc
import re
import time
import datetime
import xml.etree.ElementTree as ET

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.svm import LinearSVC

import xgboost as xgb
import lightgbm as lgbm

pd.set_option("display.max_columns",None)


# In[ ]:


train_labels=pd.read_csv("train_labels.csv")
print(train_labels.shape)
train_labels


# In[ ]:


plt.xticks(rotation='90')
column_name='RD_party_A_Fitch_long_term_trigger_method'
print(train_labels[column_name].value_counts())
sns.countplot(train_labels[column_name])
plt.show()


# In[ ]:





# In[ ]:


filenames=[name for name in os.listdir("data/") if ".xml" in name]
features=pd.DataFrame()
c=0
features['File_Name']=["NotFound"]*len(filenames)
features['Text']=0
for i,filename in enumerate(filenames):
#     print(filename,":")RD_party_A_SnP_short_term_trigger_method
    tree=ET.parse("data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        print(filename)
        features.loc[i,'File_Name']="19_bk"
    else:
        features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
#     target_child=
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"text"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if "moody" in elem.text.lower():
#                 print(elem.text)
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
#     print(datia)
    features.loc[i,'Text']=data

features=features.merge(right=train_labels[['File_Name',column_name]],how='left',on=['File_Name'])
print(features.shape)
features.head()


# In[ ]:


features.iloc[1,1]


# In[ ]:


def fitch_checker(x):
    if "fitch" in x.lower():
        return 1
    else:
        return 0

def length(x):
    return len(x.split())

features['fitch_checker']=features['Text'].apply(fitch_checker)
features['length']=features['Text'].apply(length)

features.head()


# In[ ]:





# In[ ]:


encoder={"NotFound":0,"below":1,"at":2}
decoder={0:"NotFound",1:"below",2:"at"}

features[column_name]=features[column_name].map(encoder)

features

tfidf=TfidfVectorizer(lowercase=True,stop_words='english',max_features=1000)
tfidf.fit(features['Text'])

X=tfidf.transform(features['Text']).toarray()
y=features[column_name].values
print(X.shape,y.shape)


# In[ ]:


y[y==0].shape[0]/y.shape[0]


# In[ ]:


X=np.hstack((X,features[['fitch_checker','length']]))
print(X.shape,y.shape)


# In[ ]:


X_train,X_val,y_train,y_val=train_test_split(X,y,stratify=y,test_size=0.25,random_state=42)

print("Training Shape",X_train.shape,y_train.shape)
print("Validation Shape",X_val.shape,y_val.shape)


# In[ ]:


lr=LogisticRegression(solver="liblinear",multi_class='ovr')
lr.fit(X_train,y_train)

print("Training Score",lr.score(X_train,y_train))
print("Validation Score",lr.score(X_val,y_val))


# In[ ]:


xgbclf=xgb.XGBClassifier()

xgbclf.fit(X_train,y_train)

print("Training Score",xgbclf.score(X_train,y_train))
print("Validation Score",xgbclf.score(X_val,y_val))


# In[ ]:


svc=LinearSVC()

svc.fit(X_train,y_train)

print("Training Score",svc.score(X_train,y_train))
print("Validation Score",svc.score(X_val,y_val))


# In[ ]:


test_filenames=[name for name in os.listdir("public_test_data/") if ".xml" in name]
test_features=pd.DataFrame()
c=0
test_features['File_Name']=["NotFound"]*len(test_filenames)
test_features['Text']=0
for i,filename in enumerate(test_filenames):
#     print(filename,":")RD_party_A_SnP_short_term_trigger_method
    tree=ET.parse("public_test_data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        print(filename)
        test_features.loc[i,'File_Name']="19_bk"
    else:
        test_features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"text"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if "moody" in elem.text.lower():
#                 print(elem.text)
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
#     print(datia)
    test_features.loc[i,'Text']=data

print(test_features.shape)
test_features.head()


# In[ ]:


test_features['fitch_checker']=test_features['Text'].apply(fitch_checker)
test_features['length']=test_features['Text'].apply(length)
print(test_features.shape)
test_features.head()


# In[ ]:


X_test=tfidf.transform(test_features['Text'].values).toarray()
X_test=np.hstack((X_test,test_features[['fitch_checker','length']]))
print(X_test.shape)


# In[ ]:


sub=pd.DataFrame()
sub['File_Name']=test_features['File_Name']
sub[column_name]=pd.Series(lr.predict(X_test)).map(decoder)
sub.head()


# In[ ]:


sns.countplot(sub[column_name])


# In[ ]:


sub.to_csv("public test predictions/"+column_name+".csv",index=False)


# rd party A fitch short term trigger
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import gc
import re
import time
import datetime
import xml.etree.ElementTree as ET

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

import xgboost as xgb
import lightgbm as lgbm

pd.set_option("display.max_columns",None)


# In[2]:


train_labels=pd.read_csv("train_labels.csv")
print(train_labels.shape)
train_labels


# In[3]:


plt.xticks(rotation='90')
column_name='RD_party_A_Fitch_short_term_trigger_method'
print(train_labels[column_name].value_counts())
sns.countplot(train_labels[column_name])
plt.show()


# In[ ]:





# In[4]:


filenames=[name for name in os.listdir("data/") if ".xml" in name]
features=pd.DataFrame()
c=0
features['File_Name']=["NotFound"]*len(filenames)
features['Text']=0
for i,filename in enumerate(filenames):
#     print(filename,":")RD_party_A_SnP_short_term_trigger_method
    tree=ET.parse("data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        print(filename)
        features.loc[i,'File_Name']="19_bk"
    else:
        features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
#     target_child=
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"text"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if "moody" in elem.text.lower():
#                 print(elem.text)
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
#     print(datia)
    features.loc[i,'Text']=data

features=features.merge(right=train_labels[['File_Name',column_name]],how='left',on=['File_Name'])
print(features.shape)
features.head()


# In[5]:


features.iloc[1,1]


# In[6]:


def fitch_checker(x):
    if "fitch" in x.lower():
        return 1
    else:
        return 0

def length(x):
    return len(x.split())

features['fitch_checker']=features['Text'].apply(fitch_checker)
features['length']=features['Text'].apply(length)

features.head()


# In[ ]:





# In[7]:


encoder={"NotFound":0,"below":1}
decoder={0:"NotFound",1:"below"}

features[column_name]=features[column_name].map(encoder)

features

tfidf=TfidfVectorizer(lowercase=True,stop_words='english',max_features=1000)
tfidf.fit(features['Text'])

X=tfidf.transform(features['Text']).toarray()
y=features[column_name].values
print(X.shape,y.shape)


# In[8]:


y[y==0].shape[0]/y.shape[0]


# In[9]:


X=np.hstack((X,features[['fitch_checker','length']]))
print(X.shape,y.shape)


# In[10]:


X_train,X_val,y_train,y_val=train_test_split(X,y,stratify=y,test_size=0.25,random_state=42)

print("Training Shape",X_train.shape,y_train.shape)
print("Validation Shape",X_val.shape,y_val.shape)


# In[11]:


lr=LogisticRegression(solver="liblinear")
lr.fit(X_train,y_train)

print("Training Score",lr.score(X_train,y_train))
print("Validation Score",lr.score(X_val,y_val))


# In[12]:


xgbclf=xgb.XGBClassifier()

xgbclf.fit(X_train,y_train)

print("Training Score",xgbclf.score(X_train,y_train))
print("Validation Score",xgbclf.score(X_val,y_val))


# In[13]:


test_filenames=[name for name in os.listdir("public_test_data/") if ".xml" in name]
test_features=pd.DataFrame()
c=0
test_features['File_Name']=["NotFound"]*len(test_filenames)
test_features['Text']=0
for i,filename in enumerate(test_filenames):
#     print(filename,":")RD_party_A_SnP_short_term_trigger_method
    tree=ET.parse("public_test_data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        print(filename)
        test_features.loc[i,'File_Name']="19_bk"
    else:
        test_features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"text"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if "moody" in elem.text.lower():
#                 print(elem.text)
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
#     print(datia)
    test_features.loc[i,'Text']=data

print(test_features.shape)
test_features.head()


# In[14]:


test_features['fitch_checker']=test_features['Text'].apply(fitch_checker)
test_features['length']=test_features['Text'].apply(length)
print(test_features.shape)
test_features.head()


# In[15]:


X_test=tfidf.transform(test_features['Text'].values).toarray()
X_test=np.hstack((X_test,test_features[['fitch_checker','length']]))
print(X_test.shape)


# In[16]:


sub=pd.DataFrame()
sub['File_Name']=test_features['File_Name']
sub[column_name]=pd.Series(xgbclf.predict(X_test)).map(decoder)
sub.head()


# In[17]:


sns.countplot(sub[column_name])


# In[18]:


sub.to_csv("public test predictions/"+column_name+".csv",index=False)


# # moody long term trigger method
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import gc
import re
import time
import datetime
import xml.etree.ElementTree as ET

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

import xgboost as xgb
import lightgbm as lgbm

pd.set_option("display.max_columns",None)


# In[2]:


train_labels=pd.read_csv("train_labels.csv")
print(train_labels.shape)
train_labels


# In[3]:


plt.xticks(rotation='90')
column_name='RD_party_A_moody_long_term_trigger_method'
print(train_labels[column_name].value_counts())
sns.countplot(train_labels[column_name])
plt.show()


# In[4]:


correction={"Below":"below","below":"below","at":"at","At":"at","NotFound":"NotFound"}
train_labels[column_name]=train_labels[column_name].map(correction)


# In[5]:


filenames=[name for name in os.listdir("data/") if ".xml" in name]
features=pd.DataFrame()
c=0
features['File_Name']=["NotFound"]*len(filenames)
features['Text']=0
for i,filename in enumerate(filenames):
#     print(filename,":")
    tree=ET.parse("data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        print(filename)
        features.loc[i,'File_Name']="19_bk"
    else:
        features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"text"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if "moody" in elem.text.lower():
#                 print(elem.text)
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
#     print(datia)
    features.loc[i,'Text']=data

features=features.merge(right=train_labels[['File_Name',column_name]],how='left',on=['File_Name'])
print(features.shape)
features.head()


# In[6]:


features.iloc[1,1]


# In[7]:


def moody_checker(x):
    if "moody" in x.lower():
        return 1
    else:
        return 0
    
def length(x):
    return len(x.split())

features['moody_checker']=features['Text'].apply(moody_checker)
features['length']=features['Text'].apply(length)

features.head()


# In[8]:


encoder={"NotFound":0,"below":1,"at":2}
decoder={0:"NotFound",1:"below",2:"at"}

features[column_name]=features[column_name].map(encoder)

features

tfidf=TfidfVectorizer(lowercase=True,stop_words='english',max_features=1000)
tfidf.fit(features['Text'])

X=tfidf.transform(features['Text']).toarray()
y=features[column_name].values
print(X.shape,y.shape)


# In[9]:


y[y==0].shape[0]/y.shape[0]


# In[10]:


X=np.hstack((X,features[['moody_checker','length']]))
print(X.shape,y.shape)


# In[11]:


X_train,X_val,y_train,y_val=train_test_split(X,y,stratify=y,test_size=0.25,random_state=42)

print("Training Shape",X_train.shape,y_train.shape)
print("Validation Shape",X_val.shape,y_val.shape)


# In[12]:


lr=LogisticRegression(solver="liblinear",multi_class='ovr')
lr.fit(X_train,y_train)

print("Training Score",lr.score(X_train,y_train))
print("Validation Score",lr.score(X_val,y_val))


# In[13]:


xgbclf=xgb.XGBClassifier()

xgbclf.fit(X_train,y_train)

print("Training Score",xgbclf.score(X_train,y_train))
print("Validation Score",xgbclf.score(X_val,y_val))


# In[14]:


test_filenames=[name for name in os.listdir("public_test_data/") if ".xml" in name]
test_features=pd.DataFrame()
c=0
test_features['File_Name']=["NotFound"]*len(test_filenames)
test_features['Text']=0
for i,filename in enumerate(test_filenames):
#     print(filename,":")
    tree=ET.parse("public_test_data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        print(filename)
        test_features.loc[i,'File_Name']="19_bk"
    else:
        test_features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"text"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if "moody" in elem.text.lower():
#                 print(elem.text)
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
#     print(datia)
    test_features.loc[i,'Text']=data

print(test_features.shape)
test_features.head()


# In[15]:


test_features['moody_checker']=test_features['Text'].apply(moody_checker)
test_features['length']=test_features['Text'].apply(length)
print(test_features.shape)
test_features.head()


# In[16]:


X_test=tfidf.transform(test_features['Text'].values).toarray()
X_test=np.hstack((X_test,test_features[['moody_checker','length']]))
print(X_test.shape)


# In[17]:


sub=pd.DataFrame()
sub['File_Name']=test_features['File_Name']
sub[column_name]=pd.Series(lr.predict(X_test)).map(decoder)
sub.head()


# In[18]:


sns.countplot(sub[column_name])


# In[19]:


sub.to_csv("public test predictions/"+column_name+".csv",index=False)

## moody short term trigger 

#!/usr/bin/env python
# coding: utf-8

# In[1]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import gc
import re
import time
import datetime
import xml.etree.ElementTree as ET

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

import xgboost as xgb
import lightgbm as lgbm

pd.set_option("display.max_columns",None)


# In[2]:


train_labels=pd.read_csv("train_labels.csv")
print(train_labels.shape)
train_labels


# In[3]:


plt.xticks(rotation='90')
column_name='RD_party_A_moody_short_term_trigger_method'
print(train_labels[column_name].value_counts())
sns.countplot(train_labels[column_name])
plt.show()

filenames=[name for name in os.listdir("data/") if ".xml" in name]
features=pd.DataFrame()
c=0
features['File_Name']=["NotFound"]*len(filenames)
features['Text']=0
for i,filename in enumerate(filenames):
#     print(filename,":")
    tree=ET.parse("data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        print(filename)
        features.loc[i,'File_Name']="19_bk"
    else:
        features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"text"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if "moody" in elem.text.lower():
#                 print(elem.text)
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
#     print(datia)
    features.loc[i,'Text']=data

features=features.merge(right=train_labels[['File_Name',column_name]],how='left',on=['File_Name'])
print(features.shape)
features.head()


# In[5]:


features.iloc[1,1]


# In[6]:


def moody_checker(x):
    if "moody" in x.lower():
        return 1
    else:
        return 0
    
def length(x):
    return len(x.split())

features['moody_checker']=features['Text'].apply(moody_checker)
features['length']=features['Text'].apply(length)

features.head()


# In[7]:


encoder={"NotFound":0,"below":1}
decoder={0:"NotFound",1:"below"}

features[column_name]=features[column_name].map(encoder)

features

tfidf=TfidfVectorizer(lowercase=True,stop_words='english',max_features=1000)
tfidf.fit(features['Text'])

X=tfidf.transform(features['Text']).toarray()
y=features[column_name].values
print(X.shape,y.shape)


# In[8]:


y[y==0].shape[0]/y.shape[0]


# In[9]:


X=np.hstack((X,features[['moody_checker','length']]))
print(X.shape,y.shape)


# In[10]:


X_train,X_val,y_train,y_val=train_test_split(X,y,stratify=y,test_size=0.25,random_state=42)

print("Training Shape",X_train.shape,y_train.shape)
print("Validation Shape",X_val.shape,y_val.shape)


# In[11]:


lr=LogisticRegression(solver="liblinear")
lr.fit(X_train,y_train)

print("Training Score",lr.score(X_train,y_train))
print("Validation Score",lr.score(X_val,y_val))


# In[12]:


xgbclf=xgb.XGBClassifier()

xgbclf.fit(X_train,y_train)

print("Training Score",xgbclf.score(X_train,y_train))
print("Validation Score",xgbclf.score(X_val,y_val))


# In[13]:


test_filenames=[name for name in os.listdir("public_test_data/") if ".xml" in name]
test_features=pd.DataFrame()
c=0
test_features['File_Name']=["NotFound"]*len(test_filenames)
test_features['Text']=0
for i,filename in enumerate(test_filenames):
#     print(filename,":")
    tree=ET.parse("public_test_data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        print(filename)
        test_features.loc[i,'File_Name']="19_bk"
    else:
        test_features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"text"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if "moody" in elem.text.lower():
#                 print(elem.text)
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
#     print(datia)
    test_features.loc[i,'Text']=data

print(test_features.shape)
test_features.head()


# In[14]:


test_features['moody_checker']=test_features['Text'].apply(moody_checker)
test_features['length']=test_features['Text'].apply(length)
print(test_features.shape)
test_features.head()


# In[15]:


X_test=tfidf.transform(test_features['Text'].values).toarray()
X_test=np.hstack((X_test,test_features[['moody_checker','length']]))
print(X_test.shape)


# In[16]:


sub=pd.DataFrame()
sub['File_Name']=test_features['File_Name']
sub[column_name]=pd.Series(lr.predict(X_test)).map(decoder)
sub.head()


# In[17]:


sns.countplot(sub[column_name])


# In[18]:


sub.to_csv("public test predictions/"+column_name+".csv",index=False)

## snp long term

#!/usr/bin/env python
# coding: utf-8

# In[1]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import gc
import re
import time
import datetime
import xml.etree.ElementTree as ET

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

import xgboost as xgb
import lightgbm as lgbm

pd.set_option("display.max_columns",None)


# In[2]:


train_labels=pd.read_csv("train_labels.csv")
print(train_labels.shape)
train_labels


# In[3]:


plt.xticks(rotation='90')
column_name='RD_party_A_SnP_long_term_trigger_method'
print(train_labels[column_name].value_counts())
sns.countplot(train_labels[column_name])
plt.show()


# In[4]:


correction={"Below":"below","below":"below","at":"at","At":"at","NotFound":"NotFound"}
train_labels[column_name]=train_labels[column_name].map(correction)


# In[5]:


filenames=[name for name in os.listdir("data/") if ".xml" in name]
features=pd.DataFrame()
c=0
features['File_Name']=["NotFound"]*len(filenames)
features['Text']=0
for i,filename in enumerate(filenames):
#     print(filename,":")RD_party_A_SnP_short_term_trigger_method
    tree=ET.parse("data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        print(filename)
        features.loc[i,'File_Name']="19_bk"
    else:
        features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"text"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if "moody" in elem.text.lower():
#                 print(elem.text)
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
#     print(datia)
    features.loc[i,'Text']=data

features=features.merge(right=train_labels[['File_Name',column_name]],how='left',on=['File_Name'])
print(features.shape)
features.head()


# In[6]:


features.iloc[1,1]


# In[7]:


def snp_checker(x):
    if 'snp' in x.lower():
        return 1
    else:
        return 0

def length(x):
    return len(x.split())

features['snp_checker']=features['Text'].apply(snp_checker)
features['length']=features['Text'].apply(length)


# In[8]:


encoder={"NotFound":0,"below":1,"at":2}
decoder={0:"NotFound",1:"below",2:"at"}

features[column_name]=features[column_name].map(encoder)

features

tfidf=TfidfVectorizer(lowercase=True,stop_words='english',max_features=1000)
tfidf.fit(features['Text'])

X=tfidf.transform(features['Text']).toarray()
y=features[column_name].values
print(X.shape,y.shape)


# In[9]:


y[y==0].shape[0]/y.shape[0]


# In[10]:


X=np.hstack((X,features[['snp_checker','length']]))
print(X.shape,y.shape)


# In[11]:


X_train,X_val,y_train,y_val=train_test_split(X,y,stratify=y,test_size=0.25,random_state=42)

print("Training Shape",X_train.shape,y_train.shape)
print("Validation Shape",X_val.shape,y_val.shape)


# In[12]:


lr=LogisticRegression(solver="liblinear",multi_class='ovr')
lr.fit(X_train,y_train)

print("Training Score",lr.score(X_train,y_train))
print("Validation Score",lr.score(X_val,y_val))


# In[13]:


xgbclf=xgb.XGBClassifier()

xgbclf.fit(X_train,y_train)

print("Training Score",xgbclf.score(X_train,y_train))
print("Validation Score",xgbclf.score(X_val,y_val))


# In[14]:


test_filenames=[name for name in os.listdir("public_test_data/") if ".xml" in name]
test_features=pd.DataFrame()
c=0
test_features['File_Name']=["NotFound"]*len(test_filenames)
test_features['Text']=0
for i,filename in enumerate(test_filenames):
#     print(filename,":")
    tree=ET.parse("public_test_data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        print(filename)
        test_features.loc[i,'File_Name']="19_bk"
    else:
        test_features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"text"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if "moody" in elem.text.lower():
#                 print(elem.text)
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
#     print(datia)
    test_features.loc[i,'Text']=data

print(test_features.shape)
test_features.head()


# In[15]:


test_features['snp_checker']=test_features['Text'].apply(snp_checker)
test_features['length']=test_features['Text'].apply(length)
print(test_features.shape)
test_features.head()


# In[16]:


X_test=tfidf.transform(test_features['Text'].values).toarray()
X_test=np.hstack((X_test,test_features[['snp_checker','length']]))
print(X_test.shape)


# In[17]:


sub=pd.DataFrame()
sub['File_Name']=test_features['File_Name']
sub[column_name]=pd.Series(lr.predict(X_test)).map(decoder)
sub.head()


# In[18]:


sns.countplot(sub[column_name])


# In[19]:


sub.to_csv("public test predictions/"+column_name+".csv",index=False)

## snp short term trigger method
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import gc
import re
import time
import datetime
import xml.etree.ElementTree as ET

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

import xgboost as xgb
import lightgbm as lgbm

pd.set_option("display.max_columns",None)


# In[2]:


train_labels=pd.read_csv("train_labels.csv")
print(train_labels.shape)
train_labels


# In[3]:


plt.xticks(rotation='90')
column_name='RD_party_A_SnP_short_term_trigger_method'
print(train_labels[column_name].value_counts())
sns.countplot(train_labels[column_name])
plt.show()


# In[ ]:





# In[4]:


filenames=[name for name in os.listdir("data/") if ".xml" in name]
features=pd.DataFrame()
c=0
features['File_Name']=["NotFound"]*len(filenames)
features['Text']=0
for i,filename in enumerate(filenames):
#     print(filename,":")RD_party_A_SnP_short_term_trigger_method
    tree=ET.parse("data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        print(filename)
        features.loc[i,'File_Name']="19_bk"
    else:
        features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"text"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if "moody" in elem.text.lower():
#                 print(elem.text)
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
#     print(datia)
    features.loc[i,'Text']=data

features=features.merge(right=train_labels[['File_Name',column_name]],how='left',on=['File_Name'])
print(features.shape)
features.head()


# In[5]:


features.iloc[1,1]


# In[6]:


def snp_checker(x):
    if 'snp' in x.lower():
        return 1
    else:
        return 0

def length(x):
    return len(x.split())

features['snp_checker']=features['Text'].apply(snp_checker)
features['length']=features['Text'].apply(length)


# In[7]:


encoder={"NotFound":0,"below":1}
decoder={0:"NotFound",1:"below"}

features[column_name]=features[column_name].map(encoder)

features

tfidf=TfidfVectorizer(lowercase=True,stop_words='english',max_features=1000)
tfidf.fit(features['Text'])

X=tfidf.transform(features['Text']).toarray()
y=features[column_name].values
print(X.shape,y.shape)


# In[8]:


y[y==0].shape[0]/y.shape[0]


# In[9]:


X=np.hstack((X,features[['snp_checker','length']]))
print(X.shape,y.shape)


# In[10]:


X_train,X_val,y_train,y_val=train_test_split(X,y,stratify=y,test_size=0.25,random_state=42)

print("Training Shape",X_train.shape,y_train.shape)
print("Validation Shape",X_val.shape,y_val.shape)


# In[11]:


lr=LogisticRegression(solver="liblinear")
lr.fit(X_train,y_train)

print("Training Score",lr.score(X_train,y_train))
print("Validation Score",lr.score(X_val,y_val))


# In[12]:


xgbclf=xgb.XGBClassifier()

xgbclf.fit(X_train,y_train)

print("Training Score",xgbclf.score(X_train,y_train))
print("Validation Score",xgbclf.score(X_val,y_val))


# In[13]:


test_filenames=[name for name in os.listdir("public_test_data/") if ".xml" in name]
test_features=pd.DataFrame()
c=0
test_features['File_Name']=["NotFound"]*len(test_filenames)
test_features['Text']=0
for i,filename in enumerate(test_filenames):
#     print(filename,":")
    tree=ET.parse("public_test_data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        print(filename)
        test_features.loc[i,'File_Name']="19_bk"
    else:
        test_features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"text"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if "moody" in elem.text.lower():
#                 print(elem.text)
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
#     print(datia)
    test_features.loc[i,'Text']=data

print(test_features.shape)
test_features.head()


# In[14]:


test_features['snp_checker']=test_features['Text'].apply(snp_checker)
test_features['length']=test_features['Text'].apply(length)
print(test_features.shape)
test_features.head()


# In[15]:


X_test=tfidf.transform(test_features['Text'].values).toarray()
X_test=np.hstack((X_test,test_features[['snp_checker','length']]))
print(X_test.shape)


# In[16]:


sub=pd.DataFrame()
sub['File_Name']=test_features['File_Name']
sub[column_name]=pd.Series(lr.predict(X_test)).map(decoder)
sub.head()


# In[17]:


sns.countplot(sub[column_name])


# In[18]:


sub.to_csv("public test predictions/"+column_name+".csv",index=False)


## tc bespoke

#!/usr/bin/env python
# coding: utf-8

# In[1]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import gc
import re
import time
import datetime
import xml.etree.ElementTree as ET


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import xgboost as xgb
import lightgbm as lgbm

pd.set_option("display.max_columns",None)


# In[2]:


train_labels=pd.read_csv("train_labels.csv")
print(train_labels.shape)
train_labelstrain_labels=pd.read_csv("train_labels.csv")
print(train_labels.shape)
train_labels


# In[3]:


train_labels.loc[train_labels['File_Name']=='19_bk']


# In[4]:


train_labels['TC_currency'].value_counts()/train_labels.shape[0]


# In[5]:


train_labels['TC_currency'].unique()


# In[6]:


filenames=[name for name in os.listdir("data/") if ".xml" in name]
features=pd.DataFrame()
features['File_Name']=["NotFound"]*len(filenames)
features['if_selected by the Non-defaulting Party']=0
features['if_selected by the Non-defaulting']=0
features['if_non-Affected Party']=0
features['if_not the Affected Party']=0
for i,filename in enumerate(filenames):
#     print(filename+" :")
    tree=ET.parse("data/"+filename)
    root=tree.getroot()
    # appending the file name
    if filename=="19_BK.xml":
        print(filename)
        features.loc[i,'File_Name']="19_bk"
    else:
        features.loc[i,'File_Name']=filename.split(".")[0]
    
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"formatting"):
#         print(child.text)
        if "selected by the Non-defaulting Party" in child.text:
            features.loc[i,'if_selected by the Non-defaulting Party']=1
        
        if "selected by the Non-defaulting" in child.text:
            features.loc[i,'if_selected by the Non-defaulting']=1
            
        if "non-Affected Party" in child.text:
            features.loc[i,'if_non-Affected Party']=1
            
        if "not the Affected Party" in child.text:
            features.loc[i,'if_not the Affected Party']=1
            
features=features.merge(right=train_labels[['File_Name','TC_Bespoke']],how='left',on=['File_Name'])

print(features.shape)
display(features.head(7))

encoder={"NotFound":0,"To be selected by the Non-defaulting party or non-affected party":1}
decoder=dict(zip(encoder.values(),encoder.keys()))


# In[7]:


X=features.iloc[:,1:-1].values
y=features['TC_Bespoke'].map(encoder).values

print(X.shape,y.shape)


# In[8]:


X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.25,random_state=42)

print("Training Shape",X_train.shape,y_train.shape)
print("Validation Shape",X_val.shape,y_val.shape)


# In[9]:


lr=LogisticRegression(solver='liblinear')
lr.fit(X_train,y_train)

print("Training Score :",lr.score(X_train,y_train))
print("Validation Score :",lr.score(X_val,y_val))


# In[10]:


test_filenames=[name for name in os.listdir("public_test_data/") if ".xml" in name]
test_features=pd.DataFrame()
test_features['File_Name']=["NotFound"]*len(test_filenames)
test_features['if_selected by the Non-defaulting Party']=0
test_features['if_selected by the Non-defaulting']=0
test_features['if_non-Affected Party']=0
test_features['if_not the Affected Party']=0

for i,filename in enumerate(test_filenames):
#     print(filename+" :")
    tree=ET.parse("public_test_data/"+filename)
    root=tree.getroot()
    
    # appending the file name
    if filename=="19_BK.xml":
        print(filename)
        test_features.loc[i,'File_Name']="19_bk"
    else:
        test_features.loc[i,'File_Name']=filename.split(".")[0]
    
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"formatting"):
#         print(child.text)
        if "selected by the Non-defaulting Party" in child.text:
            test_features.loc[i,'if_selected by the Non-defaulting Party']=1
        
        if "selected by the Non-defaulting" in child.text:
            test_features.loc[i,'if_selected by the Non-defaulting']=1
            
        if "non-Affected Party" in child.text:
            test_features.loc[i,'if_non-Affected Party']=1
            
        if "not the Affected Party" in child.text:
            test_features.loc[i,'if_not the Affected Party']=1

print(test_features.shape)
display(test_features.head(7))


# In[11]:


X_test=test_features.iloc[:,1:].values
print(X_test.shape)


# In[12]:


sub=pd.DataFrame()
sub['File_Name']=test_features['File_Name']
sub['TC_Bespoke']=pd.Series(lr.predict(X_test)).map(decoder)
sub.head()


# In[13]:


sns.countplot(sub['TC_Bespoke'])


# In[14]:


sub.to_csv("public test predictions/TC_Bespoke.csv",index=False)

# ca dispute resolution

#!/usr/bin/env python
# coding: utf-8

# In[1]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import gc
import re
import time
import datetime
import xml.etree.ElementTree as ET

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

import xgboost as xgb
import lightgbm as lgbm

pd.set_option("display.max_columns",None)


# In[2]:


train_labels=pd.read_csv("train_labels.csv")
print(train_labels.shape)
train_labels


# In[3]:


plt.xticks(rotation='90')
column_name='CA_dispute_resolution '
print(train_labels[column_name].value_counts())
sns.countplot(train_labels[column_name])
plt.show()


# In[4]:


correction={"Yes":"yes","YES":"yes","yes":"yes","No":"no","NO":"no","no":"no","NotFound":"no"}
train_labels[column_name]=train_labels[column_name].map(correction)


# In[5]:


filenames=[name for name in os.listdir("data/") if ".xml" in name]
features=pd.DataFrame()
c=0
features['File_Name']=["NotFound"]*len(filenames)
features['Text']=0
for i,filename in enumerate(filenames):
#     print(filename,":")RD_party_A_SnP_short_term_trigger_method
    tree=ET.parse("data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        print(filename)
        features.loc[i,'File_Name']="19_bk"
    else:
        features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"par"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if "calculation agent" in elem.text.lower():
#                 print(elem.text)
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
    features.loc[i,'Text']=data

features=features.merge(right=train_labels[['File_Name',column_name]],how='left',on=['File_Name'])
print(features.shape)
features.head()


# In[6]:


encoder={"yes":1,"no":0}
decoder={1:"yes",0:"no"}

tfidf=TfidfVectorizer(lowercase=True,stop_words='english',max_features=1000)
tfidf.fit(features['Text'])

X=tfidf.transform(features['Text']).toarray()
y=features[column_name].map(encoder).values
print(X.shape,y.shape)


# In[7]:


y[y==0].shape[0]/y.shape[0]


# In[8]:


X_train,X_val,y_train,y_val=train_test_split(X,y,stratify=y,test_size=0.25,random_state=42)

print("Training Shape",X_train.shape,y_train.shape)
print("Validation Shape",X_val.shape,y_val.shape)


# In[9]:


lr=LogisticRegression(solver="liblinear")
lr.fit(X_train,y_train)

print("Training Score",lr.score(X_train,y_train))
print("Validation Score",lr.score(X_val,y_val))


# In[10]:


xgbclf=xgb.XGBClassifier()

xgbclf.fit(X_train,y_train)

print("Training Score",xgbclf.score(X_train,y_train))
print("Validation Score",xgbclf.score(X_val,y_val))


# In[11]:

test_filenames=[name for name in os.listdir("public_test_data/") if ".xml" in name]
test_features=pd.DataFrame()
c=0
test_features['File_Name']=["NotFound"]*len(test_filenames)
test_features['Text']=0
for i,filename in enumerate(test_filenames):
#     print(filename,":")
    tree=ET.parse("public_test_data/"+filename)
    root=tree.getroot()
    if filename=="19_BK.xml":
        print(filename)
        test_features.loc[i,'File_Name']="19_bk"
    else:
        test_features.loc[i,'File_Name']=filename.split(".")[0]
    
    c=0
    for child in root.iter(tag=root.tag.split("}")[0]+"}"+"par"):
#         print(child.tag)
        for elem in child.iter(tag=child.tag.split("}")[0]+"}"+"formatting"):
            if "calculation agent" in elem.text.lower():
#                 print(elem.text)
                target_child=child
                c=1
                break
        if c==1:
            break
    
    data=""
    for child in target_child.iter(tag=target_child.tag.split("}")[0]+"}"+"formatting"):
        data=data+child.text
    
    test_features.loc[i,'Text']=data

print(test_features.shape)
test_features.head()


# In[12]:


X_test=tfidf.transform(test_features['Text'].values)
print("Testing Shape",X_test.shape)


# In[13]:


sub=pd.DataFrame()
sub['File_Name']=test_features['File_Name']
sub[column_name]=pd.Series(xgbclf.predict(X_test)).map(decoder)
sub.head()


# In[14]:


sns.countplot(sub[column_name])


# In[15]:


sub.to_csv("public test predictions/CA_dispute_resolution.csv",index=False)
print("Done")




# final merger

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os


test_filenames=os.listdir("public test predictions/")

final=pd.DataFrame()

for index,filename in enumerate(os.listdir("public test predictions/")):
    if filename!='.ipynb_checkpoints':
        print(filename)
        temp=pd.read_csv("public test predictions/"+filename)
        if index==0:
            final=temp
        else:
            final=final.merge(temp,on=['File_Name'],how='left')
        
print(final.shape)
final.head()

final.to_csv("sai_final(3).csv",index=False)


(249, 30)
NotFound                                                                                               194
third party in case of default                                                                          18
non defaulting party in case of default                                                                 14
substitution calculation agent in case of dispute                                                        7
substitution calculation agent in case of default                                                        3
other in case of dispute                                                                                 2
substitution calculation agent in case of dispute;other in case of dispute                               2
substitution calculation agent in case of default;third party in case of default                         1
non defaulting party in case of default;third party in case of default                                   1
substitution calculation ag

C:\Users\dutta\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Training Score 0.989247311827957
Validation Score 0.8253968253968254
(140, 2)
Testing Shape (140, 629)
Done
(249, 30)
No          118
no           53
NO           48
Yes          21
yes           6
YES           2
NotFound      1
Name: CA_dispute_resolution , dtype: int64
19_BK.xml
(249, 3)
(249, 629) (249,)
Training Shape (186, 629) (186,)
Validation Shape (63, 629) (63,)
Training Score 0.8817204301075269
Validation Score 0.8888888888888888
Training Score 0.978494623655914
Validation Score 0.9206349206349206
(140, 2)
Testing Shape (140, 629)
(249, 30)
19_BK.xml
(249, 3)
(249, 364) (249,)
Training Shape (186, 364) (186,)
Validation Shape (63, 364) (63,)
Training Score 0.8817204301075269
Validation Score 0.873015873015873
Training Score 0.9731182795698925
Validation Score 0.8571428571428571
(140, 2)
Testing Shape (140, 364)
Done
(249, 30)
NotFound    142
below       100
at            7
Name: RD_party_A_Fitch_long_term_trigger_method, dtype: int64


<Figure size 640x480 with 1 Axes>

19_BK.xml
(249, 3)
(249, 1000) (249,)
(249, 1002) (249,)
Training Shape (186, 1002) (186,)
Validation Shape (63, 1002) (63,)
Training Score 0.9193548387096774
Validation Score 0.9047619047619048
Training Score 0.9731182795698925
Validation Score 0.9365079365079365
Training Score 0.6236559139784946
Validation Score 0.6031746031746031


...

(249, 4) (249,)
Training Shape (186, 4) (186,)
Validation Shape (63, 4) (63,)
Training Score : 0.8924731182795699
Validation Score : 0.9206349206349206
(140, 5)


,File_Name,if_selected by the Non-defaulting Party,if_selected by the Non-defaulting,if_non-Affected Party,if_not the Affected Party
0,105_bk,0,0,0,0
1,109_bk,0,0,1,0
2,10_sh,0,0,0,0
3,11_sh,0,0,0,0
4,12_sh,0,0,0,0
5,13_sh,0,0,0,0
6,14_sh,1,1,0,1


(140, 4)
(249, 30)
No          118
no           53
NO           48
Yes          21
yes           6
YES           2
NotFound      1
Name: CA_dispute_resolution , dtype: int64


<Figure size 640x480 with 1 Axes>

19_BK.xml
(249, 3)
(249, 629) (249,)
Training Shape (186, 629) (186,)
Validation Shape (63, 629) (63,)
Training Score 0.8817204301075269
Validation Score 0.8888888888888888
Training Score 0.978494623655914
Validation Score 0.9206349206349206
(140, 2)
Testing Shape (140, 629)
Done
CA_dispute_resolution.csv
CA_fallback_default_dispute.csv
governing law.csv
RD_party_A_Fitch_long_term_trigger_method.csv
RD_party_A_Fitch_short_term_trigger_method.csv
RD_party_A_moody_long_term_trigger_method.csv
RD_party_A_moody_short_term_trigger_method.csv
RD_party_A_SnP_long_term_trigger_method.csv
RD_party_A_SnP_short_term_trigger_method.csv
TC_Bespoke.csv
(140, 11)


In [2]:
final.shape

(140, 11)

In [3]:
final.head(2)

,File_Name,CA_dispute_resolution,CA_fallback_default_dispute,governing law,RD_party_A_Fitch_long_term_trigger_method,RD_party_A_Fitch_short_term_trigger_method,RD_party_A_moody_long_term_trigger_method,RD_party_A_moody_short_term_trigger_method,RD_party_A_SnP_long_term_trigger_method,RD_party_A_SnP_short_term_trigger_method,TC_Bespoke
0,105_bk,no,substitution calculation agent in case of dispute,Laws of the State of New York,below,NotFound,below,NotFound,below,NotFound,NotFound
1,109_bk,no,NotFound,English Law,below,NotFound,below,NotFound,below,NotFound,To be selected by the Non-defaulting party or ...
